In [3]:
import torch
import torch


class DummySpecAug:
    def __init__(self, mask_token, patch_height=1, max_mask_pct=0.2):
        self.mask_token = mask_token
        self.patch_height = patch_height
        self.max_mask_pct = max_mask_pct

    def apply_specaugment_mask(self, X, X_len, num_masks=2):
        """
        Fully vectorized SpecAugment-style time masking (no loops).
        Args:
            X: (B, P, D)
            X_len: (B,) in timepoints
        Returns:
            X_masked: (B, P, D), mask: (B, P)
        """
        B, P, D = X.shape
        device = X.device

        valid_lens = (X_len // self.patch_height).to(device)
        max_mask_lens = (self.max_mask_pct * valid_lens).clamp(min=1).long()

        B_rep = B * num_masks
        valid_lens_rep = valid_lens.repeat_interleave(num_masks)
        max_mask_lens_rep = max_mask_lens.repeat_interleave(num_masks)

        t = (torch.rand(B_rep, device=device) * max_mask_lens_rep.float()).floor().long() + 1
        max_start = (valid_lens_rep - t + 1).clamp(min=1)
        t0 = (torch.rand(B_rep, device=device) * max_start.float()).floor().long()

        # Build flattened mask indices
        arange = torch.arange(P, device=device).unsqueeze(0)  # (1, P)
        t0_exp = t0.unsqueeze(1)                              # (B_rep, 1)
        t1_exp = (t0 + t).unsqueeze(1)                        # (B_rep, 1)
        mask_chunks = (arange >= t0_exp) & (arange < t1_exp)  # (B_rep, P)

        # Now gather the flat indices to write into (B, P)
        batch_idx = torch.arange(B, device=device).repeat_interleave(num_masks)  # (B_rep,)
        patch_idx = mask_chunks.nonzero(as_tuple=False)  # (N_masked, 2): [mask_row, patch_col]
        b_indices = batch_idx[patch_idx[:, 0]]           # Map B_rep index → actual batch index
        p_indices = patch_idx[:, 1]                      # patch index

        # Set those positions to True in the full mask
        mask = torch.zeros(B, P, dtype=torch.bool, device=device)
        mask[b_indices, p_indices] = True

        # Apply mask token
        X_masked = X.clone()
        X_masked[mask] = self.mask_token

        return X_masked, mask


# --------------------------
# Create dummy input
B, P, D = 2, 12, 4  # batch, patches, dim
X = torch.randn(B, P, D)
X_len = torch.tensor([12, 10])  # valid lengths (in timepoints)

# Define mask token and module
mask_token = torch.tensor([-99.0] * D)
specaug = DummySpecAug(mask_token=mask_token, patch_height=1, max_mask_pct=0.3)

# Apply
X_masked, mask = specaug.apply_specaugment_mask(X, X_len, num_masks=2)

# --------------------------
# Inspect the result
for b in range(B):
    print(f"\nSample {b}:")
    print("Original:")
    print(X[b])
    print("Masked:")
    print(X_masked[b])
    print("Mask positions:", mask[b].nonzero(as_tuple=True)[0].tolist())



Sample 0:
Original:
tensor([[-1.5667, -0.8848, -0.0823, -1.0963],
        [-0.2630, -1.9768, -1.0156,  1.9896],
        [-0.4240,  0.9485, -2.6145, -1.4046],
        [ 0.6807,  0.2921, -0.8919,  1.0357],
        [-0.3071,  0.4782, -0.0400,  0.0223],
        [-0.0794, -0.3305,  0.8095, -1.4395],
        [ 0.2538, -0.8903,  0.3576, -0.7388],
        [-1.0737,  0.1499, -0.8428,  0.6036],
        [ 2.2194,  1.8892, -0.3977, -0.5240],
        [ 0.2136, -0.3794, -1.0174,  1.5791],
        [-0.7023,  0.5966,  0.8818,  0.3092],
        [-1.2846,  0.4203, -2.1773,  0.3069]])
Masked:
tensor([[-9.9000e+01, -9.9000e+01, -9.9000e+01, -9.9000e+01],
        [-9.9000e+01, -9.9000e+01, -9.9000e+01, -9.9000e+01],
        [-9.9000e+01, -9.9000e+01, -9.9000e+01, -9.9000e+01],
        [ 6.8074e-01,  2.9214e-01, -8.9190e-01,  1.0357e+00],
        [-3.0712e-01,  4.7820e-01, -4.0047e-02,  2.2303e-02],
        [-7.9409e-02, -3.3050e-01,  8.0954e-01, -1.4395e+00],
        [ 2.5378e-01, -8.9029e-01,  3.5758e-01